# Nurse scheduling in CPMpy

In [1]:
from cpmpy import *

def simple_nurse_rostering(num_nurses, shifts_per_day, num_days):

    # Define the variables
    roster_matrix = intvar(0, num_nurses - 1, shape=(shifts_per_day, num_days))

    # Define the constraints
    model = Model()

    # Constraint: Each shift in a day must be assigned to a different nurse
    for day in range(num_days):
        model += AllDifferent(roster_matrix[:, day])

    # Constraint: The last shift of a day cannot have the same nurse as the first shift of the next day
    for day in range(num_days - 1):
        model += (roster_matrix[shifts_per_day - 1, day] != roster_matrix[0, day + 1])
    
    # Make sure fair allocation of shifts
    min_nb_shifts = min([sum(roster_matrix == n) for n in range(num_nurses)])
    max_nb_shifts = max([sum(roster_matrix == n) for n in range(num_nurses)])
    model.minimize(max_nb_shifts - min_nb_shifts)
    
    return model, (roster_matrix,)


In [2]:
# Example data
num_nurses = 10
shifts_per_day = 5
num_days = 7

In [3]:
model, (roster,) = simple_nurse_rostering(num_nurses, shifts_per_day, num_days)

In [4]:
import pandas as pd

def weekday(i):
    if i % 7 == 0: return "Monday"
    if i % 7 == 1: return "Tuesday"
    if i % 7 == 2: return "Wednesday"
    if i % 7 == 3: return "Thursday"
    if i % 7 == 4: return "Friday"
    if i % 7 == 5: return "Saturday"
    if i % 7 == 6: return "Sunday"
    

def make_pretty_roster(styler):

    styler.set_caption("Nurse roster")
    styler.format_index(lambda v: f"Shift {v+1}")
    styler.format_index(weekday, axis="columns")
    styler.background_gradient(axis=None, vmin=1, vmax=num_nurses, cmap="Pastel1") 
    styler.set_properties(**{'text-align': 'center'})
    display(styler)

In [5]:
# Solve the problem
solution = model.solve()

if solution:
    # Display the nurse roster
    pd.DataFrame(roster.value()+1).style.pipe(make_pretty_roster)
else:
    print("No solution found.")
    print(model)


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Shift 1,7,10,2,4,3,10,2
Shift 2,6,4,5,1,6,4,5
Shift 3,9,7,7,9,1,7,3
Shift 4,5,1,10,3,9,1,4
Shift 5,3,8,6,2,2,8,8
